In [69]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver

In [70]:
load_dotenv()

llm= ChatGroq(model = "openai/gpt-oss-120b")


In [71]:
class JokeState(TypedDict):
    
    topic: str
    joke: str
    explanation: str

In [72]:
def generate_joke(state: JokeState):

    prompt = f'generate a joke on the topic {state["topic"]}'
    response = llm.invoke(prompt).content

    return {'joke': response}

In [73]:
def generate_explanation(state: JokeState):

    prompt = f'write an explanation for the joke - {state["joke"]}'
    response = llm.invoke(prompt).content

    return {'explanation': response}

In [74]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)

In [75]:
config1 = {"configurable": {"thread_id": "1"}}
workflow.invoke({'topic':'pizza'}, config=config1)

{'topic': 'pizza',
 'joke': 'Why did the pizza apply for a job?  \n\nBecause it heard the company was looking for “well‑rounded” candidates who could really *deliver* under pressure! 🍕😄',
 'explanation': '**Why the joke works**\n\n1. **The set‑up:**  \n   *“Why did the pizza apply for a job?”*  \n   This primes the listener to expect a punchline that ties a piece of food to a workplace situation—something that, on the surface, seems absurd.\n\n2. **The punchline:**  \n   *“Because it heard the company was looking for ‘well‑rounded’ candidates who could really *deliver* under pressure!”*  \n\n3. **Breaking down the wordplay**\n\n   | Phrase in the punchline | Literal meaning (pizza) | Figurative meaning (job interview) |\n   |--------------------------|--------------------------|-------------------------------------|\n   | **well‑rounded** | A pizza is literally a round, circular food. | In hiring lingo, “well‑rounded” describes a candidate with a broad set of skills and experiences. |\

In [76]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza apply for a job?  \n\nBecause it heard the company was looking for “well‑rounded” candidates who could really *deliver* under pressure! 🍕😄', 'explanation': '**Why the joke works**\n\n1. **The set‑up:**  \n   *“Why did the pizza apply for a job?”*  \n   This primes the listener to expect a punchline that ties a piece of food to a workplace situation—something that, on the surface, seems absurd.\n\n2. **The punchline:**  \n   *“Because it heard the company was looking for ‘well‑rounded’ candidates who could really *deliver* under pressure!”*  \n\n3. **Breaking down the wordplay**\n\n   | Phrase in the punchline | Literal meaning (pizza) | Figurative meaning (job interview) |\n   |--------------------------|--------------------------|-------------------------------------|\n   | **well‑rounded** | A pizza is literally a round, circular food. | In hiring lingo, “well‑rounded” describes a candidate with a broad set of skills 

In [77]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza apply for a job?  \n\nBecause it heard the company was looking for “well‑rounded” candidates who could really *deliver* under pressure! 🍕😄', 'explanation': '**Why the joke works**\n\n1. **The set‑up:**  \n   *“Why did the pizza apply for a job?”*  \n   This primes the listener to expect a punchline that ties a piece of food to a workplace situation—something that, on the surface, seems absurd.\n\n2. **The punchline:**  \n   *“Because it heard the company was looking for ‘well‑rounded’ candidates who could really *deliver* under pressure!”*  \n\n3. **Breaking down the wordplay**\n\n   | Phrase in the punchline | Literal meaning (pizza) | Figurative meaning (job interview) |\n   |--------------------------|--------------------------|-------------------------------------|\n   | **well‑rounded** | A pizza is literally a round, circular food. | In hiring lingo, “well‑rounded” describes a candidate with a broad set of skills

In [78]:
config2 = {"configurable": {"thread_id": "2"}}
workflow.invoke({'topic':'pasta'}, config=config2)

{'topic': 'pasta',
 'joke': 'Why did the spaghetti get a promotion?\n\nBecause it always knew how to *work* its *pasta*-tude and never got tangled up in office drama! 🍝😄',
 'explanation': '**Explanation of the joke**\n\n> **Why did the spaghetti get a promotion?**  \n> **Because it always knew how to *work* its *pasta*-tude and never got tangled up in office drama! 🍝😄**\n\n---\n\n### 1. The set‑up (the question)\n\nThe question sounds like a typical “Why did X get a promotion?” office‑humor setup. We expect the punchline to involve a work‑related quality (e.g., “hard work,” “team spirit,” “leadership”).  \n\n### 2. The punchline – the wordplay\n\nThe answer contains **three intertwined puns** that play on the nature of spaghetti and on common business‑jargon:\n\n| Phrase in the punchline | What it sounds like / refers to | How it ties to spaghetti |\n|--------------------------|--------------------------------|--------------------------|\n| **“work its *pasta*-tude”** | A mash‑up of **

In [79]:
workflow.get_state(config2)

StateSnapshot(values={'topic': 'pasta', 'joke': 'Why did the spaghetti get a promotion?\n\nBecause it always knew how to *work* its *pasta*-tude and never got tangled up in office drama! 🍝😄', 'explanation': '**Explanation of the joke**\n\n> **Why did the spaghetti get a promotion?**  \n> **Because it always knew how to *work* its *pasta*-tude and never got tangled up in office drama! 🍝😄**\n\n---\n\n### 1. The set‑up (the question)\n\nThe question sounds like a typical “Why did X get a promotion?” office‑humor setup. We expect the punchline to involve a work‑related quality (e.g., “hard work,” “team spirit,” “leadership”).  \n\n### 2. The punchline – the wordplay\n\nThe answer contains **three intertwined puns** that play on the nature of spaghetti and on common business‑jargon:\n\n| Phrase in the punchline | What it sounds like / refers to | How it ties to spaghetti |\n|--------------------------|--------------------------------|--------------------------|\n| **“work its *pasta*-tude”*

In [80]:
list(workflow.get_state_history(config2))

[StateSnapshot(values={'topic': 'pasta', 'joke': 'Why did the spaghetti get a promotion?\n\nBecause it always knew how to *work* its *pasta*-tude and never got tangled up in office drama! 🍝😄', 'explanation': '**Explanation of the joke**\n\n> **Why did the spaghetti get a promotion?**  \n> **Because it always knew how to *work* its *pasta*-tude and never got tangled up in office drama! 🍝😄**\n\n---\n\n### 1. The set‑up (the question)\n\nThe question sounds like a typical “Why did X get a promotion?” office‑humor setup. We expect the punchline to involve a work‑related quality (e.g., “hard work,” “team spirit,” “leadership”).  \n\n### 2. The punchline – the wordplay\n\nThe answer contains **three intertwined puns** that play on the nature of spaghetti and on common business‑jargon:\n\n| Phrase in the punchline | What it sounds like / refers to | How it ties to spaghetti |\n|--------------------------|--------------------------------|--------------------------|\n| **“work its *pasta*-tude”

### TIME TRAVEL


In [81]:

workflow.get_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f10125b-2504-6038-8001-f6cac73406fa"}})

StateSnapshot(values={}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f10125b-2504-6038-8001-f6cac73406fa'}}, metadata=None, created_at=None, parent_config=None, tasks=(), interrupts=())

In [82]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f10125b-2504-6038-8001-f6cac73406fa"}})

EmptyInputError: Received no input for __start__

In [ ]:
list(workflow.get_state_history(config1))

### UPDATING STATE

In [ ]:
workflow.update_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f10125b-2504-6038-8001-f6cac73406fa", "checkpoint_ns": ""}}, {'topic':'samosa'})

In [ ]:

list(workflow.get_state_history(config1))

In [ ]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f10125b-2504-6038-8001-f6cac73406fa"}}

_IncompleteInputError: incomplete input (1963652315.py, line 1)

In [ ]:
list(workflow.get_state_history(config1))

### FAULT TOLERANCE

In [ ]:
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import InMemorySaver
from typing import TypedDict
import time

In [ ]:
# 1. Define the state
class CrashState(TypedDict):
    input: str
    step1: str
    step2: str

In [ ]:
# 2. Define steps
def step_1(state: CrashState) -> CrashState:
    print("✅ Step 1 executed")
    return {"step1": "done", "input": state["input"]}

def step_2(state: CrashState) -> CrashState:
    print("⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)")
    time.sleep(10)  # Simulate long-running hang
    return {"step2": "done"}

def step_3(state: CrashState) -> CrashState:
    print("✅ Step 3 executed")
    return {"done": True}

In [ ]:
# 3. Build the graph
builder = StateGraph(CrashState)
builder.add_node("step_1", step_1)
builder.add_node("step_2", step_2)
builder.add_node("step_3", step_3)

builder.set_entry_point("step_1")
builder.add_edge("step_1", "step_2")
builder.add_edge("step_2", "step_3")
builder.add_edge("step_3", END)

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)

In [ ]:
try:
    print("▶️ Running graph: Please manually interrupt during Step 2...")
    graph.invoke({"input": "start"}, config={"configurable": {"thread_id": 'thread-1'}})
except KeyboardInterrupt:
    print("❌ Kernel manually interrupted (crash simulated).")

▶️ Running graph: Please manually interrupt during Step 2...
✅ Step 1 executed
⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)
✅ Step 3 executed


In [ ]:
# 6. Re-run to show fault-tolerant resume
print("\n🔁 Re-running the graph to demonstrate fault tolerance...")
final_state = graph.invoke(None, config={"configurable": {"thread_id": 'thread-1'}})
print("\n✅ Final State:", final_state)


🔁 Re-running the graph to demonstrate fault tolerance...

✅ Final State: {'input': 'start', 'step1': 'done', 'step2': 'done'}


In [ ]:
# 6. Re-run to show fault-tolerant resume
print("\n🔁 Re-running the graph to demonstrate fault tolerance...")
final_state = graph.invoke(None, config={"configurable": {"thread_id": 'thread-1'}})
print("\n✅ Final State:", final_state)
list(graph.get_state_history({"configurable": {"thread_id": 'thread-1'}}))


🔁 Re-running the graph to demonstrate fault tolerance...

✅ Final State: {'input': 'start', 'step1': 'done', 'step2': 'done'}


[StateSnapshot(values={'input': 'start', 'step1': 'done', 'step2': 'done'}, next=(), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f101249-30af-6397-8003-4fe98ca770a7'}}, metadata={'source': 'loop', 'step': 3, 'parents': {}}, created_at='2026-02-03T17:20:02.261477+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f101249-30ac-6364-8002-939c56295dfd'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'input': 'start', 'step1': 'done', 'step2': 'done'}, next=('step_3',), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f101249-30ac-6364-8002-939c56295dfd'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2026-02-03T17:20:02.260233+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f101248-d149-6542-8001-deec26066013'}}, tasks=(PregelTask(id='92c1eb9e-4989-4fea-0a52-9f1cc3cc723